In [1]:
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from modeling.train_utils import array_to_dataloader
import torch
from torch.nn import functional as F

In [2]:
class reconstruct_CNN(nn.Module):
    def __init__(self, num_neuron):
        super().__init__()
        modules = []

        hidden_dims = [16, 64, 128, 64, 16]

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride = 2,
                                       padding= 1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
        )
        self.final_layer = nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[-1],
                                       hidden_dims[-1],
                                       kernel_size=5,
                                       stride=1,
                                       padding=2,
                                       output_padding=2,
                                       dilation=5),
                    nn.BatchNorm2d(hidden_dims[-1]),
                    nn.LeakyReLU(),
                    nn.Conv2d(hidden_dims[-1], out_channels= 1,
                              kernel_size= 3, padding= 1),
                    nn.Tanh())


        self.layers = nn.Sequential(*modules)
        self.linear_input = nn.Linear(num_neuron, hidden_dims[0] * 4)

    def forward(self, x):
        x = self.linear_input(x)
        x = x.view(-1, 16, 2, 2)
        x = self.layers(x)
        x = self.final_layer(x)
        return x

In [3]:
site = 'm1s1'
train_x = np.load('../data/Processed_Tang_data/all_sites_data_prepared/pics_data/train_img_'+site+'.npy')
val_x = np.load('../data/Processed_Tang_data/all_sites_data_prepared/pics_data/val_img_'+site+'.npy')
train_y = np.load('../data/Processed_Tang_data/all_sites_data_prepared/New_response_data/trainRsp_'+site+'.npy')
val_y = np.load('../data/Processed_Tang_data/all_sites_data_prepared/New_response_data/valRsp_'+site+'.npy')
model = reconstruct_CNN(302)
train_x = np.transpose(train_x, (0, 3, 1, 2))
val_x = np.transpose(val_x, (0, 3, 1, 2))
train_loader = array_to_dataloader(train_x, train_y, batch_size=1024, shuffle=True)
val_loader = array_to_dataloader(val_x, val_y, batch_size=1024)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
criterion = F.mse_loss
device = 'cuda'

In [4]:
network = model.to(device)
losses = []
accs = []

bestloss = 200
num_epochs = 100
for e in tqdm(range(num_epochs)):

    train_losses = []
    network = network.train()
    for i, (x, y) in enumerate(train_loader):
        x = x.float().to(device)
        y = y.float().to(device)
        preds = network(y)
        loss = criterion(preds, x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
    losses.append(np.mean(train_losses))

    val_losses = []
    with torch.no_grad():
        network = network.eval()
        for i, (x, y) in enumerate(val_loader):
            x = x.float().to(device)
            y = y.float().to(device)
            preds = network(y)
            loss = criterion(preds, x)
            val_losses.append(loss.item())
    avg_loss = np.mean(val_losses)
    accs.append(avg_loss)
    if avg_loss < bestloss:
        torch.save(network.state_dict(), "direct_CNN")
        bestloss = avg_loss

    print(f'epoch {e} : train loss is {float(losses[-1])}')
    print(f'epoch {e} : val loss is   {float(accs[-1])}')

  1%|          | 1/100 [00:08<14:19,  8.68s/it]

epoch 0 : train loss is 0.04768678332295488
epoch 0 : val loss is   0.02894783206284046


  2%|▏         | 2/100 [00:14<11:51,  7.26s/it]

epoch 1 : train loss is 0.027093951919061297
epoch 1 : val loss is   0.028357338160276413


  3%|▎         | 3/100 [00:21<10:56,  6.77s/it]

...

 65%|██████▌   | 65/100 [06:45<03:40,  6.29s/it]

epoch 64 : train loss is 0.021040090578882134
epoch 64 : val loss is   0.019734041765332222


 66%|██████▌   | 66/100 [06:52<03:32,  6.25s/it]

epoch 65 : train loss is 0.021052421728039488
epoch 65 : val loss is   0.02004658244550228


KeyboardInterrupt: 

In [5]:
import matplotlib.pyplot as plt

In [6]:
sample = torch.tensor(val_y[:], dtype=torch.float).to(device)
#model.load_state_dict(torch.load('direct_CNN'))
model = model.to(device)
recon = model(sample).detach().cpu().numpy()
origin = val_x[:100]
for i, (r_img, img) in enumerate(zip(recon, origin)):
    r_img = np.reshape(r_img, (50, 50))
    img = np.reshape(img, (50, 50))
    print("newimg")
    plt.imsave(f'results/recon_{i}.png',r_img, cmap='gray')
    plt.show()
    plt.imsave(f'results/origin_{i}.png',img, cmap='gray')
    plt.show()

newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg
newimg


In [7]:
val_losses = []
accs = []
with torch.no_grad():
    network = model.eval()
    for i, (x, y) in enumerate(val_loader):
        x = x.float().to(device)
        y = y.float().to(device)
        preds = network(y)
        loss = criterion(preds, x)
        val_losses.append(loss.item())
avg_loss = np.mean(val_losses)
accs.append(avg_loss)

print(f'val loss is   {float(accs[-1])}')

val loss is   0.02003301866352558
